## 问答题
1. 支持向量机的基本思想是什么？

2. 什么是支持向量？

3. 在使用 SVM 时，缩放输入值为什么很重要？

4. SVM 分类器在对实例进行分类时能输出置信度分数吗？概率呢？

5. 你如何在 LinearSVC、SVC 和 SGDClassifier 之间进行选择？

6. 假设你已经使用 RBF 核训练了一个 SVM 分类器，但它似乎欠拟合训练集。
   你应该增大还是减小 γ（gamma）？C 呢？

7. ε 不敏感模型是什么意思？

8. 使用核技巧有什么意义？

## 编程题
1. 在葡萄酒数据集上训练SVM分类器，可以使用sklearn.datasets.load_wine()加载它。该数据集包含3个不同种植者生产的178个葡萄酒样本的化学分析：目标是训练一个分类模型，该模型能够根据葡萄酒的化学分析预测种植者。由于SVM分类器是二元分类器，将需要使用“一对全部”对所有三个类进行分类。能达到的精度是多少？

   "一对全部"可以复习 **8_sklearn做分类.ipynb**里的笔记，里面提到了用二元分类器做多分类问题

---

2. 提前预习 **10_支持向量机.ipynb** 最新更新的笔记 （把SVM分类用梯度下降实现）； 大概理解笔记后，尝试自己对照笔记 实现用梯度下降实现SVM分类

   并把自定义的SVM分类用于 iris data(鸢尾花数据)； 取花瓣长度 和 花瓣宽度特征， 分类 看是不是 分类2的花 （(iris.target == 2)

   对比下sklearn自带的SVM分类 和 自定义SVM分类 实现的分类效果

---

3. 在加州房屋数据集上训练和微调SVM回归器。可以使用原始数据集而不是 在课上使用的调整后的版本，
可以使用sklearn.datasets.fetch_california_housing()加载它。目标代表了数十万美元。
由于有超过20000个实例，SVM可能会很慢，因此对于超参数调整，应该使用更少的实例（例如2000个）来测试更多的超参数组合。最佳模型的RMSE是多少？


In [47]:
from sklearn.datasets import fetch_california_housing
import numpy as np
import pandas as pd
base_data = fetch_california_housing(as_frame=True)
data = base_data.frame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [48]:
# 没有缺失值 全部数据为数值类型

# 分层采样 特征分离
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV
# data.data
# data.target.name
data['MedInc_cut'] = pd.cut(data['MedInc'],
                               bins= [0., 1.5, 3., 4.5, 6., np.inf],
                               labels = [1,2,3,4,5])
split = StratifiedShuffleSplit(n_splits = 3, test_size = 0.2, random_state = 35)
for train_index, test_index in split.split(data, data['MedInc_cut']):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

X_train = strat_train_set.drop(base_data.target.name, axis=1)
y_train = strat_train_set[base_data.target.name]
X_test = strat_test_set.drop(base_data.target.name, axis=1)
y_test = strat_test_set[base_data.target.name]

# 丢掉辅助列
for train_index, test_index in split.split(data, data['MedInc_cut']):
    strat_train_set = data.loc[train_index].drop('MedInc_cut', axis=1)
    strat_test_set = data.loc[test_index].drop('MedInc_cut', axis=1)


# 数据归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_prepared = scaler.fit_transform(X_train)
X_test_prepared = scaler.transform(X_test)


# 限制搜寻范围 这部分来自gpt
n_search = 2000  # 搜索子集大小
rng = np.random.RandomState(42)
idx = rng.choice(len(X_train), size=min(n_search, len(X_train)), replace=False)

X_train_sub = X_train_prepared[idx]
y_train_sub = y_train.iloc[idx]



# 模型训练与调参
from sklearn.svm import SVR
SVR_model = SVR()
param_distributions = {
    'kernel': ['rbf', 'linear'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale','auto', 0.01, 0.001],
    'epsilon': [0.001, 0.1, 0.2, 0.5]}
grid_search = RandomizedSearchCV(SVR_model, param_distributions, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, n_iter = 20, n_jobs=-1, random_state=42)
grid_search.fit(X_train_sub, y_train_sub)
best_model = grid_search.best_estimator_


# 效率验证部分
from sklearn.metrics import mean_squared_error
final_predictions = best_model.predict(X_test_prepared)
final_rmse =np.sqrt(mean_squared_error(y_test, final_predictions))
best_score = grid_search.best_score_
best_cv_rmse = np.sqrt(-best_score)
print("预测值:", final_predictions[:5])
print("实际值:", y_test.values[:5])
print("finalRMSE:", final_rmse, "|", "Best CV RMSE:",best_cv_rmse)

预测值: [0.93462758 3.24924147 2.56259548 2.01626802 2.74917228]
实际值: [0.855 3.881 2.75  2.613 3.5  ]
finalRMSE: 0.623099373931964 | Best CV RMSE: 0.6645780449460803


In [50]:
# gpt给出的管道封装版本，留作模板
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 数据标准化
    ('svr', SVR())                 # SVM 回归
])

# 6. 定义超参数搜索范围（注意：使用 'svr__' 前缀）
param_distributions = {
    'svr__kernel': ['rbf', 'linear'],
    'svr__C': [0.1, 1, 10, 100],
    'svr__gamma': ['scale', 'auto', 0.01, 0.001],
    'svr__epsilon': [0.001, 0.1, 0.2, 0.5]
}

# 7. 随机搜索
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)
search.fit(X_train_sub, y_train_sub)

# 8. 最佳模型
best_model = search.best_estimator_
print("Best parameters:", search.best_params_)

# 9. 在完整测试集上评估
final_predictions = best_model.predict(X_test_prepared)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
print("Final RMSE:", final_rmse)

Best parameters: {'svr__kernel': 'rbf', 'svr__gamma': 'scale', 'svr__epsilon': 0.2, 'svr__C': 10}
Final RMSE: 0.617684925915562
